In [970]:
import pandas as pd
import numpy as np

1. Implement in python program of the following problems using Bayes Theorem. 

    a)  Of  the  students  in  the  college,  60%  of  the  students  reside  in  the  hostel  and  40%  of  the  students  are  day 
    scholars. Previous year results report that 30% of all students who stay in the hostel scored A Grade and 20% 
    of day scholars scored A grade. At the end of the year, one student is chosen at random and found that he/she 
    has an A grade. What is the probability that the student is a hosteler?  

In [971]:
p_hostel=0.6
p_day=0.4
p_A_hostel=0.3
p_A_day=0.2

p_A=p_hostel*p_A_hostel+p_day*p_A_day

p_hostel_A=p_A_hostel*p_hostel/p_A

print("p_hostel_A =",p_hostel_A)

p_hostel_A = 0.6923076923076923


b) Suppose you're testing for a rare disease, and you have the following information: 

1. The disease has a prevalence of 0.01 (1% of the population has the disease). 
2. The test is not perfect: 
3. The test correctly identifies the disease (true positive) 99% of the time (sensitivity). 
4. The test incorrectly indicates the disease (false positive) 2% of the time (1 - specificity). 

Calculate the probability of having the disease given a positive test result using Bayes' theorem.

In [972]:
p_disease=0.01
p_nodisease=1-p_disease
p_pos_disease=0.99
p_pos_nodisease=0.2

p_pos=p_pos_disease*p_disease+p_pos_nodisease*p_nodisease

p_disease_pos=p_pos_disease*p_disease/p_pos

print("p_disease_pos =",p_disease_pos)

p_disease_pos = 0.04761904761904762


2. Develop a function python code for Naïve Bayes classifier from scratch without using scikit-learn library, 
to predict whether the buyer should buy computer or not. Consider a following sample training dataset stored 
in a CSV file containing information about following buyer conditions (such as “<=30,” “medium,” “Yes,” 
and “fair”) and whether the player played golf (“Yes” or “No”).

In [973]:
def class_prob(df,y):
    p={}
    for cls in df[y].unique():
        p[cls]=(df[y]==cls).sum()/len(df)
    return p

In [974]:
def feature_prob(df,y):
    probs={}

    for cls in df[y].unique():
        probs[cls]={}
        rows=df[df[y]==cls]
        for col in df.columns[:-1]:
            probs[cls][col]={}
            for value in rows[col].unique():
                cnt=(rows[col]==value).sum()
                probs[cls][col][value]=cnt/len(rows)
    return probs

In [975]:
def predict(sample,priors,probs):
    result={}
    for label in priors:
        prob=priors[label]
        for feature,value in sample.items():
            if value in probs[label][feature]:
                prob*=probs[label][feature][value]
        result[label]=prob
    return max(result,key=lambda x:result[x])

In [976]:
df=pd.read_csv('computer.csv')

y='buys_computer'

In [977]:
priors=class_prob(df,y)
probs=feature_prob(df,y)
probs

{'no': {'age': {'<=30': 0.6, '>40': 0.4},
  'income': {'high': 0.4, 'low': 0.2, 'medium': 0.4},
  'student': {'no': 0.8, 'yes': 0.2},
  'credit_rating': {'fair': 0.4, 'excellent': 0.6}},
 'yes': {'age': {'31...40': 0.4444444444444444,
   '>40': 0.3333333333333333,
   '<=30': 0.2222222222222222},
  'income': {'high': 0.2222222222222222,
   'medium': 0.4444444444444444,
   'low': 0.3333333333333333},
  'student': {'no': 0.3333333333333333, 'yes': 0.6666666666666666},
  'credit_rating': {'fair': 0.6666666666666666,
   'excellent': 0.3333333333333333}}}

In [978]:
sample=['<=30','medium','yes','fair']
x=pd.Series(sample,index=df.columns[:-1])

In [979]:
result=predict(x,priors,probs)
result

'yes'

3. Write a Python  function  to  implement  the  Naive  Bayes  classifier  without  using  the  scikit-learn  library  for  the 
following sample training dataset stored as a .CSV file. Calculate the accuracy, precision, and recall for your train/test 
dataset. 

    a. Build a classifier that determines whether a text is about sports or not. 

    b. Determine which tag the sentence "A very close game" belongs to.

In [980]:
df=pd.read_csv('text.csv')
target='Tag'
text=df['Text']

In [981]:
def text_prob(text,target): 
    prob={}
    cls_words={}

    for sentence,cls in zip(text,df[target]):
        if cls not in cls_words:
            cls_words[cls]=0
        words=sentence.split()
        if cls not in prob:
            prob[cls]={}
        for word in words:
            if word.lower() not in prob[cls]:
                prob[cls][word.lower()]=1
            else:
                prob[cls][word.lower()]+=1
            cls_words[cls]+=1

    for cls in prob:
        for word in prob[cls]:
            prob[cls][word]/=cls_words[cls]
            
    return prob


In [982]:
priors=class_prob(df,target)
prob=text_prob(text,target)
prob

{'Sports': {'a': 0.18181818181818182,
  'great': 0.09090909090909091,
  'game': 0.18181818181818182,
  'very': 0.09090909090909091,
  'clean': 0.18181818181818182,
  'match': 0.09090909090909091,
  'but': 0.09090909090909091,
  'forgettable': 0.09090909090909091},
 'Not sports': {'the': 0.1111111111111111,
  'election': 0.2222222222222222,
  'was': 0.2222222222222222,
  'over': 0.1111111111111111,
  'it': 0.1111111111111111,
  'a': 0.1111111111111111,
  'close': 0.1111111111111111}}

In [983]:
def predict(priors,text_prob,sample):
    result={}
    words=sample.split()
    for label in priors:
        prob=priors[label]
        for word in words:
            if word in text_prob[label]:
                prob*=text_prob[label][word]
        result[label]=prob
    return max(result,key=lambda x:result[x])

In [984]:
sample="A very close game"
predict(priors,prob,sample)

'Not sports'